In [1]:
'''
READ ME:

This code can be used to pull any point spatial Dataset using a Google Places API using place search and add it to your local gdb.
You will need to input your API Key, action type, query, the output file type, and the download save location.
The code that builds the url and inserted the arguments for the cursor was written by ChatGPT.
'''

import requests
import arcpy

api_key = '' # Required input for notebook function

action_type = 'textsearch' # Required input for notebook function (other options include findplace and nearbyserach)
google_api_url = f'https://maps.googleapis.com/maps/api/place/{action_type}/json' # Assembling query url (output type can also be xml)
query = 'playgrounds in Minneapolis' # Required input for notebook function 
url = f'{google_api_url}?query={query}&key={api_key}' # Build entire url


response = requests.get(url) # Run request with the built url
data = response.json() # Format request as JSON
output_playgrounds = data['results'] # Select only the output results

In [14]:
from arcpy import env

env.workspace = r'C:\School\GIS 5571\Local\Lab1 Data\Lab1 Project\Lab1 Project.gdb' # Setting environment to project gdb
output_feature_class = 'Playgrounds' # setting feature class name
arcpy.management.CreateFeatureclass(env.workspace, output_feature_class, 'POINT', spatial_reference=arcpy.SpatialReference(4326)) # Create a new feature class for points

output_fields = [('Name', 'TEXT'), ('Address', 'TEXT')] # Setting feature class fields
for field_name, field_type in output_fields: # Add fields to the feature class
    arcpy.management.AddField(output_feature_class, field_name, field_type)

with arcpy.da.InsertCursor(output_feature_class, ['SHAPE@XY'] + [field[0] for field in output_fields]) as cursor: # Populate feature class with playgrouond data
    for place in output_playgrounds:
        x = place['geometry']['location']['lng'] # setting Long
        y = place['geometry']['location']['lat'] # setting Lat
        name = place['name'] # Setting name
        address = place['formatted_address'] # Setting address
        cursor.insertRow([(x, y), name, address]) # Inserting new point feature with information

NameError: name 'env' is not defined

In [52]:
# The following code is present if the user would like to export the new feature class
in_features = output_feature_class
out_features = r'C:\School\GIS 5571\Local\Lab1 Data\playgrounds.shp'
arcpy.conversion.ExportFeatures(in_features, out_features)

<Result 'C:\\School\\GIS 5571\\Local\\Lab1 Data\\playgrounds.shp'>